In [3]:
#R SAI RAJAJI-2017B4A70550H

import numpy as np
import re
import random
from collections import defaultdict

In [4]:
import numpy as np
import re
import random
from collections import defaultdict

stopwords = frozenset([
    "a", "about", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
    "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot",
    "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "doesn,t",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers",
    "herself", "him", "himself", "his", "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd",
    "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself",
    "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere",
    "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over",
    "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so",
    "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the",
    "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these",
    "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too",
    "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether",
    "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without",
    "would", "yet", "you", "your", "yours", "yourself", "yourselves", ""])

bad_chars = [';', ':', '!', "*", "(", ")", "\"", ".", ",", "  ", "-", "?", "$", "[", "]", "/", "\\", "'", "1", "2", "3",
             "4", "5", "6", "7", "8", "9"]
#Reading file
f = open("dataset_NB.txt", "r")
t = f.read().splitlines()
f.close()
#Shuffling
a = random.sample(t, len(t))
x = []
X = []
Y = []
for i in a:
    x.append(i[:-1])
    Y.append(i[-1])

x = [i.lower() for i in x]

for i in x:
    for j in bad_chars:
        i = i.replace(j, ' ')

    X.append(i)

train_l=0
train_r=0
av=0
#Training 7-fold
for q in range(1,8):
    spam = defaultdict()
    ham = defaultdict()
    spamt = defaultdict()
    hamt = defaultdict()
    vocab = defaultdict()
    train_r = train_l + 1000 // 7
    append_list_x = X[0:train_l]
    append_list_y = Y[0:train_l]
    trainX = X[train_r:] + append_list_x
    trainY = Y[train_r:] + append_list_y
    testX = X[train_l:train_r]
    testY = Y[train_l:train_r]
    cs, ch = 0, 0
#Calculating frequenices
    for (i, j) in zip(trainX, trainY):
        l = re.split(' ', i)
        if (j == '0'):
            cs += 1
            for k in l:
                if (k not in stopwords) and len(k) > 2:
                    if k not in spam:
                        spam[k] = 1
                    else:
                        spam[k] += 1

                    if k not in vocab:
                        vocab[k] = 1
                    else:
                        vocab[k] += 1
        else:
            ch += 1
            for k in l:

                if (k not in stopwords) and len(k) > 2:
                    if k not in ham:
                        ham[k] = 1
                    else:
                        ham[k] += 1

                    if k not in vocab:
                        vocab[k] = 1
                    else:
                        vocab[k] += 1

    # probspam and probham are probabilities of message being spam or not
    probspam = cs / (cs + ch)
    probham = 1 - probspam


    # Calculating prior probabilities

    for k, v in spam.items():
        spam[k] = spam[k] / cs

    for k, v in ham.items():
        ham[k] = ham[k] / ch

    Y_pred = []
    for i in testX:
        l = re.split(' ', i)
        pspam = probspam
        pham = probham
        for j in l:
            if j in spam:
                pspam = pspam * spam[j]
            else:
                pspam = pspam * (1 / (cs+len(vocab)))
            if j in ham:
                pham = pham * ham[j]
            else:
                pham = pham * (1 / (ch+len(vocab)))

        if pspam > pham:
            Y_pred.append('0')
        else:
            Y_pred.append('1')

    sum = 0
    for x, y in zip(testY, Y_pred):
        if x == y:
            sum += 1
    #printing accuracy
    print("FOLD",q,"accuracy",sum / np.shape(testY)[0] * 100)
    av+=(sum / np.shape(testY)[0]) * 100
    train_l = train_l + 1 + 1000 // 7
print("The average accuracy is ",av/7)





FOLD 1 accuracy 80.28169014084507
FOLD 2 accuracy 76.76056338028168
FOLD 3 accuracy 80.28169014084507
FOLD 4 accuracy 79.5774647887324
FOLD 5 accuracy 79.5774647887324
FOLD 6 accuracy 79.5774647887324
FOLD 7 accuracy 83.09859154929578
The average accuracy is  79.87927565392354
